In [124]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes
import datetime
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import Point
matplotlib.rcParams['pdf.fonttype'] = 42


%matplotlib inline 

In [47]:
df=pd.read_csv('2018_Green_Taxi_Trip_Data.csv')

In [48]:
df=df.drop(columns=['VendorID', 'store_and_fwd_flag','extra','mta_tax','improvement_surcharge','tolls_amount'])

In [49]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,ehail_fee,total_amount,payment_type,trip_type
0,01/01/2018 12:18:50 AM,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,NaN,7.3,2,1.0
1,01/01/2018 12:30:26 AM,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,NaN,15.8,2,1.0
2,01/01/2018 12:07:25 AM,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,NaN,11.3,2,1.0
3,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,NaN,-4.3,3,1.0
4,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,NaN,4.3,2,1.0


In [52]:
df=df.drop(columns=['ehail_fee'])

KeyError: "['ehail_fee'] not found in axis"

In [53]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type
0,01/01/2018 12:18:50 AM,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0
1,01/01/2018 12:30:26 AM,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0
2,01/01/2018 12:07:25 AM,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0
3,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0
4,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0


In [54]:
df['month'] = df['lpep_pickup_datetime'].str.extract('^([0-9]{1,2})/[0-9]+/[0-9]{4}')

In [55]:
df['year'] = df['lpep_pickup_datetime'].str.extract('^[0-9]{1,2}/[0-9]+/([0-9]{4}).*?')

In [56]:
df['lpep_pickup_datetime'] = pd.to_datetime(df.lpep_pickup_datetime, format='%m/%d/%Y %I:%M:%S %p')

In [57]:
df['day_of_week'] = df['lpep_pickup_datetime'].dt.day_name()

In [28]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week
0,2018-01-01 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday
1,2018-01-01 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday
2,2018-01-01 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday
3,2018-01-01 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday
4,2018-01-01 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday


In [58]:
df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].dt.strftime('%m/%d/%Y %H:%M:%S')

In [59]:
df['hour'] = df['lpep_pickup_datetime'].str.extract('^[0-9]{1,2}/[0-9]+/[0-9]{4}\s([0-9]{2}):[0-9]{2}:[0-9]{2}')

In [36]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00


In [60]:
df['hour'].value_counts()

18    610245
17    581546
19    559289
16    546914
15    515394
20    473665
14    471877
09    449119
21    435461
10    433923
12    429991
11    427465
13    426050
08    406538
22    395685
23    345345
07    270728
00    266849
01    193878
02    139010
06    135943
03    112135
04     99326
05     80927
Name: hour, dtype: int64

In [61]:
df['month'].value_counts()

03    836973
04    800125
05    797283
01    793594
02    769810
06    739352
10    710482
12    685442
07    684442
09    666877
08    666325
11    656598
Name: month, dtype: int64

In [62]:
df['year'].value_counts()

2018    8806705
2010        251
2009        140
2019         83
2008         75
2017         44
2030          2
2020          2
2081          1
Name: year, dtype: int64

In [40]:
df.loc[df['year'] == '2081']

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour
1527751,06/24/2081 17:40:37,06/24/2081 06:42:47 PM,1,93,117,1,16.95,52.0,0.0,55.96,1,1.0,06,2081,Tuesday,17


In [42]:
df['year'].value_counts()

2018    8806705
2010        251
2009        140
2019         83
2008         75
2017         44
2030          2
2020          2
2081          1
Name: year, dtype: int64

In [63]:
df1=df[df.year == '2018']

In [64]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00


In [66]:
df=df1

In [95]:
df['month'].value_counts()/8806705*100

03    9.503691
04    9.085350
05    9.053125
01    9.008988
02    8.741113
06    8.395319
10    8.067512
12    7.781832
07    7.771828
09    7.569528
08    7.566099
11    7.455615
Name: month, dtype: float64

In [94]:
df['day_of_week'].value_counts()/8806705*100

Saturday     15.908458
Friday       15.872156
Thursday     14.528691
Wednesday    13.902623
Tuesday      13.549211
Monday       13.149333
Sunday       13.089527
Name: day_of_week, dtype: float64

In [89]:
df['hour'].value_counts()/8806705*100

18    6.929277
17    6.603378
19    6.350673
16    6.210121
15    5.852143
20    5.378379
14    5.358054
09    5.099626
21    4.944483
10    4.927087
12    4.882496
11    4.853813
13    4.837734
08    4.616142
22    4.492713
23    3.920229
07    3.074056
00    3.028091
01    2.200255
02    1.578048
06    1.543528
03    1.273030
04    1.127777
05    0.918868
Name: hour, dtype: float64

In [88]:
df['RatecodeID'].value_counts()/8806705*100

1     96.973772
5      2.721029
2      0.189958
4      0.068777
3      0.044546
6      0.001158
99     0.000761
Name: RatecodeID, dtype: float64

In [87]:
df['PULocationID'].value_counts()/8806705*100

74     6.452720
75     5.751527
41     5.530661
7      4.570007
82     4.263433
166    3.822360
42     3.382037
97     2.789988
129    2.712081
95     2.680912
181    2.619493
33     2.513392
255    2.447669
244    2.373964
65     2.328306
25     2.295876
260    2.175093
66     1.884042
223    1.873073
130    1.527143
116    1.474377
145    1.247209
49     1.107463
61     1.054537
226    0.982808
43     0.961279
112    0.894546
256    0.888562
196    0.887937
52     0.871416
         ...   
148    0.001101
245    0.001067
125    0.001022
50     0.000988
144    0.000965
118    0.000942
143    0.000931
251    0.000908
88     0.000829
261    0.000818
158    0.000772
151    0.000749
214    0.000727
172    0.000647
187    0.000625
5      0.000477
232    0.000352
45     0.000307
44     0.000238
209    0.000227
30     0.000216
176    0.000216
84     0.000182
224    0.000148
109    0.000148
4      0.000136
2      0.000125
204    0.000102
99     0.000034
12     0.000023
Name: PULocationID, Leng

In [86]:
df['DOLocationID'].value_counts()/8806705*100

74     3.421439
42     3.329384
41     3.000294
7      2.977992
129    2.696639
75     2.302110
181    2.131081
166    1.915802
82     1.812085
223    1.695356
97     1.506466
61     1.496882
95     1.451860
236    1.444672
244    1.416761
116    1.375702
238    1.359226
226    1.323412
49     1.320471
260    1.283011
33     1.157209
138    1.125222
25     1.088364
17     1.086502
179    1.073682
112    1.047054
263    1.008402
152    0.935571
151    0.903721
239    0.861718
         ...   
111    0.010776
184    0.009459
58     0.008982
253    0.008743
59     0.008176
206    0.007392
115    0.006665
221    0.006597
12     0.006552
156    0.006223
23     0.006098
118    0.003952
27     0.003872
214    0.003020
245    0.002691
251    0.002657
6      0.002612
172    0.002487
187    0.002157
109    0.001930
44     0.001669
84     0.001510
5      0.001499
30     0.001238
176    0.001135
204    0.000897
2      0.000386
99     0.000125
105    0.000034
110    0.000011
Name: DOLocationID, Leng

In [93]:
df['passenger_count'].value_counts()/8806705*100

1    84.748518
2     7.739875
5     3.391768
6     1.777941
3     1.614293
4     0.583703
0     0.140473
7     0.001601
8     0.001465
9     0.000363
Name: passenger_count, dtype: float64

In [92]:
df['payment_type'].value_counts()/8806705*100

1    56.849684
2    42.467313
3     0.461989
4     0.216755
5     0.004258
Name: payment_type, dtype: float64

In [91]:
df['trip_type'].value_counts()/8806705*100

1.0    97.214940
2.0     2.784344
Name: trip_type, dtype: float64

In [90]:
df['tip_amount'].value_counts()/8806705*100

 0.00     62.991096
 1.00      2.698171
 2.00      1.990620
 1.46      0.751734
 3.00      0.749338
 1.56      0.743161
 1.36      0.724698
 1.66      0.710572
 1.76      0.673192
 1.26      0.657295
 1.86      0.627340
 1.96      0.593911
 2.06      0.556701
 1.16      0.540565
 2.16      0.516856
 2.26      0.478079
 1.70      0.475660
 1.50      0.453189
 2.36      0.446716
 2.46      0.424279
 2.56      0.399991
 1.20      0.378825
 5.00      0.377553
 2.66      0.371365
 1.06      0.370207
 2.20      0.354139
 1.95      0.353537
 2.76      0.341808
 2.45      0.320529
 2.86      0.318768
            ...    
 23.10     0.000011
 23.15     0.000011
 23.85     0.000011
 23.90     0.000011
 23.65     0.000011
 22.40     0.000011
 22.85     0.000011
 32.32     0.000011
 41.43     0.000011
 29.15     0.000011
-0.80      0.000011
 28.65     0.000011
 31.10     0.000011
 31.15     0.000011
 31.85     0.000011
 25.35     0.000011
 24.35     0.000011
 24.40     0.000011
 24.15     0.000011


In [97]:
df=df1

In [101]:
df1['fare_per_person']=df1['total_amount']/df1['passenger_count']

In [100]:
pd.options.mode.chained_assignment = None

In [102]:
df1

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,date,fare_per_person
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.00,7.30,2,1.0,01,2018,Monday,00,01/01,1.460
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.00,15.80,2,1.0,01,2018,Monday,00,01/01,3.160
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,01/01,11.300
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.00,-4.30,3,1.0,01,2018,Monday,00,01/01,-4.300
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.00,4.30,2,1.0,01,2018,Monday,00,01/01,4.300
5,01/01/2018 00:38:35,01/01/2018 01:08:50 AM,1,255,161,1,5.63,21.0,0.00,22.30,2,1.0,01,2018,Monday,00,01/01,22.300
6,01/01/2018 00:18:41,01/01/2018 12:28:22 AM,1,189,65,5,1.71,8.5,0.00,9.80,2,1.0,01,2018,Monday,00,01/01,1.960
7,01/01/2018 00:38:02,01/01/2018 12:55:02 AM,1,189,225,5,3.45,14.5,3.16,18.96,1,1.0,01,2018,Monday,00,01/01,3.792
8,01/01/2018 00:05:02,01/01/2018 12:18:35 AM,1,129,82,1,1.61,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,01/01,11.300
9,01/01/2018 00:35:23,01/01/2018 12:42:07 AM,1,226,7,1,1.87,7.5,0.00,8.80,2,1.0,01,2018,Monday,00,01/01,8.800


In [103]:
df1['date'] = df1['lpep_pickup_datetime'].str.extract('^([0-9]{1,2}/[0-9]+)/[0-9]{4}\s([0-9]{2}):[0-9]{2}:[0-9]{2}')

In [106]:
df1

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,date,fare_per_person
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.00,7.30,2,1.0,01,2018,Monday,00,01/01,1.460
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.00,15.80,2,1.0,01,2018,Monday,00,01/01,3.160
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,01/01,11.300
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.00,-4.30,3,1.0,01,2018,Monday,00,01/01,-4.300
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.00,4.30,2,1.0,01,2018,Monday,00,01/01,4.300
5,01/01/2018 00:38:35,01/01/2018 01:08:50 AM,1,255,161,1,5.63,21.0,0.00,22.30,2,1.0,01,2018,Monday,00,01/01,22.300
6,01/01/2018 00:18:41,01/01/2018 12:28:22 AM,1,189,65,5,1.71,8.5,0.00,9.80,2,1.0,01,2018,Monday,00,01/01,1.960
7,01/01/2018 00:38:02,01/01/2018 12:55:02 AM,1,189,225,5,3.45,14.5,3.16,18.96,1,1.0,01,2018,Monday,00,01/01,3.792
8,01/01/2018 00:05:02,01/01/2018 12:18:35 AM,1,129,82,1,1.61,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,01/01,11.300
9,01/01/2018 00:35:23,01/01/2018 12:42:07 AM,1,226,7,1,1.87,7.5,0.00,8.80,2,1.0,01,2018,Monday,00,01/01,8.800


In [108]:
df1['date'].value_counts()

02/02    33953
03/10    32872
04/14    32453
03/09    31990
03/16    31799
02/24    31596
05/05    31491
03/17    31414
03/03    31365
02/09    31354
02/03    31272
01/13    31203
03/24    31202
02/16    31028
02/17    30906
02/23    30896
04/28    30855
01/26    30818
02/10    30735
04/20    30726
01/27    30599
03/23    30512
01/12    30462
03/02    30449
01/20    30276
01/19    30262
03/31    29968
04/07    29965
03/15    29940
05/12    29832
         ...  
09/02    19579
08/27    19579
11/04    19520
12/27    19410
09/10    19384
08/19    19334
09/11    19331
12/24    19314
08/13    19248
12/02    19231
08/22    19138
03/07    18877
11/24    18825
12/23    18789
08/20    18676
11/12    18514
11/15    18226
10/08    17948
05/28    17606
12/30    17295
07/04    16975
12/26    16929
11/22    16706
11/25    16598
11/23    16277
09/03    15988
08/21    15866
03/21    11981
12/25    11681
01/04     6519
Name: date, Length: 365, dtype: int64

In [117]:
df1['fare_per_person'].value_counts()

 7.800000      238157
 6.800000      235146
 7.300000      233975
 8.300000      220753
 8.800000      212335
 6.300000      211970
 9.300000      188765
 5.800000      186142
 9.800000      180441
 10.300000     158434
 10.800000     150668
 5.300000      137404
 11.300000     134523
 11.800000     131042
 12.300000     116326
 12.800000     112025
 13.300000     100091
 13.800000      96578
 4.800000       91319
 14.300000      86322
 14.800000      85246
 15.300000      79898
 15.800000      75168
 16.300000      65780
 16.800000      64481
 17.300000      57973
 17.800000      57108
 8.760000       54339
 9.360000       53571
 4.300000       53485
                ...  
-35.300000          1
-35.200000          1
 12.182000          1
 107.600000         1
 107.400000         1
 107.100000         1
 107.900000         1
 107.100000         1
 12.318000          1
 104.600000         1
 104.150000         1
 104.100000         1
 104.350000         1
-74.000000          1
 108.35000

In [118]:
df1['prange']=df1['fare_per_person']

In [119]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,date,fare_per_person,prange
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00,01/01,1.46,1.46
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00,01/01,3.16,3.16
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00,01/01,11.30,11.30
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00,01/01,-4.30,-4.30
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00,01/01,4.30,4.30


In [125]:
bins = [0, 10, 20, 30, 40, 50, 100, np.inf]
names = ['<10', '10-20', '20-30', '30-40', '40-50','50-100','100+']
# df['prange'] = pd.cut(df['fare_per_person'], bins, labels=names)

d = dict(enumerate(names, 1))

df1['prange'] = np.vectorize(d.get)(np.digitize(df['fare_per_person'], bins))

In [126]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,date,fare_per_person,prange
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00,01/01,1.46,<10
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00,01/01,3.16,<10
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00,01/01,11.30,10-20
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00,01/01,-4.30,None
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00,01/01,4.30,<10


In [127]:
bins = [0, 10, 20, 30, 40, 50, 100, np.inf]
names = ['<10', '10-20', '20-30', '30-40', '40-50','50-100','100+']
# df['prange'] = pd.cut(df['fare_per_person'], bins, labels=names)

d = dict(enumerate(names, 1))

df1['farerange'] = np.vectorize(d.get)(np.digitize(df['total_amount'], bins))

In [130]:
df['prange'].value_counts()/8806705*100

<10       46.365718
10-20     33.934269
20-30     10.675820
30-40      4.302460
50-100     2.276425
40-50      1.971555
None       0.402250
100+       0.071502
Name: prange, dtype: float64

In [129]:
df['farerange'].value_counts()

<10       3459155
10-20     3361698
20-30     1076730
30-40      438499
50-100     236503
40-50      203335
None        23059
100+         7726
Name: farerange, dtype: int64